This code is from Linda Punt for the paper "On Efficient Operation of VPP"

It is my personal perception that I don't understand many of the decisions taken in preprocessing this data.
For example:
* Removing negative prices from Day Ahead

In [21]:
import pandas as pd
import numpy as np
from scipy.stats import truncnorm, rv_discrete
import logging
from datetime import timedelta
import matplotlib.pyplot as plt
import yaml

In [22]:
def read_yaml(file):
    with open(file, 'r') as yml_file:
        config = yaml.load(yml_file, Loader=yaml.FullLoader)
        return config

In [23]:
def process_columns_da(dt):
    # Change column names
    #dt.columns = ['date', 'price_da', 'country']
    dt.columns = ['date', 'price_da']
    #dt.drop(columns='country', inplace=True)

    # Remove - prices
    dt = dt[dt.price_da != '-']
    dt['price_da'] = dt['price_da'].astype(float)
    dt = dt.dropna()

    # Change dates
    dt['date'] = dt.date.str.split('-').str[0]
    dt['date'] = pd.to_datetime(dt.date, format='%d.%m.%Y %H:%M ')

    # Get month and hour column
    dt['id_hour'] = dt.date.dt.hour

    print("Process DA")
    display(dt.describe())
    return dt

In [24]:
def process_columns_imbalance(dt):
    # Change dates
    dt['date'] = dt.apply(lambda row: pd.to_datetime(f'{row["Date"]} {row["period_until"]}',
                                                        format='%m/%d/%Y %H:%M'), axis=1)
    dt.rename(columns={'Consume': 'price_imbalance', 'Regulation state': 'regulation_state'},
              inplace=True)
    print("Process imbalance")
    display(dt.describe())
    return dt

In [25]:
def process_columns_mid(dt):
    dt['date'] = dt.apply(lambda row: pd.to_datetime(f'{row["Date"]} {row["Time"]}',
                                                        format='%m/%d/%Y %H:%M'), axis=1)
    dt['date'] = dt['date'].dt.round('15min')
    dt.rename(columns={'Mid_price_upward': 'mid_price'}, inplace=True)
    dt = dt.drop_duplicates(subset='date')
    print("Process mid")
    display(dt.describe())
    return dt

In [26]:
def process_dt(dt_da, dt_imbalance, dt_midprice):
    
    #quarter_hours = pd.DataFrame(data={'minutes': [0, 15, 30, 45], 'key': 1})
    quarter_hours = pd.DataFrame(data={'minutes': [0], 'key': 1})
    
    # Get quarter hour
    dt_da['key'] = 1
    dt = pd.merge(dt_da[['date', 'price_da', 'key']], quarter_hours, on='key')
    dt['date'] = dt.apply(lambda row: row['date'].replace(minute=row['minutes']), axis=1)

    # Get imbalance prices
    dt = pd.merge(dt, dt_imbalance[['date', 'price_imbalance', 'regulation_state']], on='date')
    dt = pd.merge(dt, dt_midprice[['date', 'mid_price']], on='date', how='left')
    dt['price_imbalance'] = np.where(dt['regulation_state'] == 2, dt['mid_price'], dt['price_imbalance'])

    # Add hour of the day and day of the year
    dt['hour'] = dt.date.dt.hour
    dt['day_year'] = dt.date.dt.dayofyear

    dt.drop(columns=['date', 'mid_price', 'regulation_state', 'key'], inplace=True)

    return dt

In [27]:
def simulate_dt(d_config, dt):
    # Make dataframe with hours and dates
    days = pd.DataFrame(data={'date': pd.date_range(start=pd.to_datetime(d_config['general']['start_date'],
                                                                         format='%d-%m-%Y'),
                                                    periods=d_config['general']['n_days']),
                              'key': 1})
    hours = pd.DataFrame(data={'hour': list(range(0, 24)),
                               'key': 1})
    
    #quarter_hours = pd.DataFrame(data={'minutes': [0, 15, 30, 45], 'key': 1})
    quarter_hours = pd.DataFrame(data={'minutes': [0], 'key': 1})
    
    data_simulated = pd.merge(days, hours, on='key')
    data_simulated = pd.merge(data_simulated, quarter_hours, on='key')
    data_simulated['date'] = data_simulated.apply(
        lambda row: row['date'].replace(hour=row['hour'], minute=row['minutes']), axis=1)
    data_simulated['day_year'] = data_simulated.date.dt.dayofyear

    # Merge dataframes
    data_simulated = pd.merge(data_simulated, dt, on=['day_year', 'minutes', 'hour'], how='left')

    # Multiply by error term
    data_simulated['error'] = 1 #+ np.random.normal(0, 0.2, data_simulated.shape[0]) # Commented out by Javier
    data_simulated['price_da'] = data_simulated['price_da'] * data_simulated['error']
    data_simulated['price_imbalance'] = data_simulated['price_imbalance'] * data_simulated['error']

    return data_simulated[['date', 'price_da', 'price_imbalance']]

In [28]:
def run(d_config):
    logging.info('Running d_process_pv_prices.py')
    dt_da = pd.read_csv('./data/Linda/day-ahead_prices_2019.csv')
    dt_imbalance = pd.read_csv('./data/Linda/imbalance_prices_2019.csv')
    dt_midprice = pd.read_csv('./data/Linda/mid_price_2019.csv')

    # Rename columns
    dt_da = process_columns_da(dt_da)
    dt_imbalance = process_columns_imbalance(dt_imbalance)
    dt_midprice = process_columns_mid(dt_midprice)

    # Process data
    dt_processed = process_dt(dt_da, dt_imbalance, dt_midprice)
    print("Processed data")
    display(dt_processed.describe())

    # Simulate data
    dt_simulated = simulate_dt(d_config, dt_processed)

    return dt_simulated, dt_processed

In [29]:
#if __name__ == "__main__":
#import os
#os.chdir('..')
run_config = read_yaml('./data/Linda/config.yml')
data_config = read_yaml('./data/Linda/data_config.yml')

final_dt, processed_dt = run(data_config)

Process DA


,price_da,id_hour
count,8760.000000,8760.000000
mean,41.195801,11.500000
std,11.278806,6.922582
min,-9.020000,0.000000
25%,34.100000,5.750000
50%,39.700000,11.500000
75%,47.392500,17.250000
max,121.460000,23.000000


Process imbalance


,PTE,To regulate up,To regulate down,Incentive component,price_imbalance,Feed,regulation_state
count,35040.000000,19971.000000,16910.000000,35040.0,35040.000000,35040.000000,35040.000000
mean,48.500457,63.592258,18.612762,0.0,42.554082,40.391060,0.223459
std,27.712496,62.519171,27.792309,0.0,52.003945,51.227613,1.022846
min,1.000000,0.000000,-487.650000,0.0,-487.650000,-487.650000,-1.000000
25%,24.750000,32.600000,18.000000,0.0,25.530000,24.470000,-1.000000
50%,48.500000,40.630000,24.500000,0.0,32.160000,31.110000,1.000000
75%,72.250000,57.105000,30.200000,0.0,42.540000,40.900000,1.000000
max,100.000000,936.120000,179.010000,0.0,936.120000,936.120000,2.000000


Process mid


,Sequence_number,IGCCContribution_up,IGCCContribution_down,To regulate up,To regulate down,To regulate up_reserve,To regulate down_reserve,Emergency capacity,Highest_price_upward,mid_price,Lowest_price_downward
count,35038.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35038.000000,15042.000000,35036.000000,12000.000000
mean,721.527370,35.396935,38.903043,39.567845,23.787733,0.155954,0.002569,0.002454,55.355570,30.478970,22.961194
std,415.693744,75.558363,85.568180,75.163300,55.291123,2.354327,0.256207,0.049483,54.065099,8.065918,21.774535
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.310000,-429.950000
25%,369.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.290000,25.780000,21.057500
50%,729.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,38.080000,29.760000,26.080000
75%,1085.250000,31.000000,32.000000,45.000000,14.000000,0.000000,0.000000,0.000000,50.190000,35.190000,31.780000
max,1494.000000,555.000000,770.000000,559.000000,541.000000,75.000000,35.000000,1.000000,936.120000,107.400000,179.010000


Processed data


,price_da,minutes,price_imbalance,hour,day_year
count,8760.000000,8760.0,8760.000000,8760.000000,8760.000000
mean,41.193841,0.0,40.972971,11.500000,183.047945
std,11.280501,0.0,51.920979,6.922582,105.377493
min,-9.020000,0.0,-239.900000,0.000000,1.000000
25%,34.100000,0.0,24.400000,5.750000,92.000000
50%,39.700000,0.0,30.975000,11.500000,183.000000
75%,47.392500,0.0,41.225000,17.250000,274.000000
max,121.460000,0.0,936.120000,23.000000,365.000000


In [30]:
final_dt

,date,price_da,price_imbalance
0,2018-07-01 00:00:00,28.98,23.89
1,2018-07-01 01:00:00,26.23,28.34
2,2018-07-01 02:00:00,25.02,26.09
3,2018-07-01 03:00:00,24.87,23.65
4,2018-07-01 04:00:00,25.90,28.47
...,...,...,...
46661,2023-10-26 19:00:00,37.37,43.51
46662,2023-10-26 20:00:00,35.77,17.57
46663,2023-10-26 21:00:00,35.05,21.65
46664,2023-10-26 22:00:00,31.82,34.19


In [31]:
final_dt = final_dt.set_index('date')

In [32]:
final_dt = final_dt[~final_dt.index.duplicated(keep='first')]

In [33]:
#Replace NaN values with mean

final_dt['price_da'] = final_dt['price_da'].fillna(final_dt['price_da'].mean())
final_dt['price_imbalance'] = final_dt['price_imbalance'].fillna(final_dt['price_imbalance'].mean())

In [34]:
#final_dt.to_csv('J:/Thesis code/thesis_code_saidur/Code_Yashar_Project/Yashar_Collab_Linda_Code/Linda NEW/synthetic_price_data_saidur.csv',index=True)

In [35]:
final_dt.describe()

,price_da,price_imbalance
count,46656.000000,46656.000000
mean,40.988010,40.780491
std,11.197233,51.924871
min,-9.020000,-239.900000
25%,33.950000,24.210000
50%,39.505000,30.610000
75%,47.180000,40.950000
max,121.460000,936.120000


In [36]:
final_dt_cp = final_dt.copy()
final_dt_cp["year"] = final_dt_cp.index.year
final_dt_2019 = final_dt_cp[final_dt_cp["year"] == 2019]
final_dt_2019 = final_dt_2019.drop(columns=["year"])
final_dt_2019.describe()

,price_da,price_imbalance
count,8760.000000,8760.000000
mean,41.197656,40.980199
std,11.277584,51.918739
min,-9.020000,-239.900000
25%,34.100000,24.407500
50%,39.710000,30.985000
75%,47.392500,41.225000
max,121.460000,936.120000


In [37]:
processed_dt.describe()

,price_da,minutes,price_imbalance,hour,day_year
count,8760.000000,8760.0,8760.000000,8760.000000,8760.000000
mean,41.193841,0.0,40.972971,11.500000,183.047945
std,11.280501,0.0,51.920979,6.922582,105.377493
min,-9.020000,0.0,-239.900000,0.000000,1.000000
25%,34.100000,0.0,24.400000,5.750000,92.000000
50%,39.700000,0.0,30.975000,11.500000,183.000000
75%,47.392500,0.0,41.225000,17.250000,274.000000
max,121.460000,0.0,936.120000,23.000000,365.000000


In [38]:
asdasdasdad

NameError: name 'asdasdasdad' is not defined

In [ ]:
#price_dt_2018 = final_dt

In [ ]:
# REMEMBER TO CHANGE THE START DATE IN D_CONFIG FILE FIRST!!!!!!

In [ ]:
price_dt_2018.head()

In [ ]:
price_dt_2020.head()

In [ ]:
price_dt_2019.head()

In [ ]:
price_dataset_saidur_FINAL = pd.concat([price_dt_2018, price_dt_2019, price_dt_2020])

In [ ]:
#price_dataset_saidur_FINAL.to_csv('price_dataset_saidur_FINAL.csv', index=False)